In [1]:
import cvxpy as cp
import numpy as np
import os
import math
import pandas as pd
import datetime as dt
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
import warnings
import time
from scipy.stats import norm
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Any, Optional

### types_file

In [2]:
class PrimalSolution:
    def __init__(self,
                 support: List[int],
                 U: List[float],
                 value: float,
                 offset: List[float],
                 slope_1: np.ndarray,
                 slope_2: np.ndarray,
                 R_sample: List[int],):
        self.support = support
        self.U = U
        self.value = value
        self.offset = offset
        self.slope_1 = slope_1
        self.slope_2 = slope_2
        self.R_sample = R_sample

### gapcalculate function

In [3]:
def gapcalculate(obj, bnd):
    gap = abs(obj - bnd) / obj
    return gap

### 主问题求解(cut的生成)

In [4]:
def Allocation_2D(
        iter, NP, N_Bus, ESS_candidate, R_bounds, C_bounds, obj_2nd, lambda_2nd, mu_2nd, 
        previous_rating, previous_cap, Fixed_cost, Power_rating_cost, Energy_capacity_cost, C_rate=1):

    print("start solving master problem...")

    # Unfolding data:
    R_min = R_bounds[0]
    R_max = R_bounds[1]
    C_min = C_bounds[0]
    C_max = C_bounds[1]

    # Adding variables for MILP
    U = cp.Variable((N_Bus), boolean=True) # Location for ESS solutions
    Cap_U = cp.Variable(N_Bus)  # Continuous variable representing the maximum energy storage capacity at each node.
    Rating_U = cp.Variable(N_Bus)  # Continuous variable representing the maximum energy storage rating at each node.
    alpha = cp.Variable() # using for benders decomposition

    #Values that need to be calculated
    non_candidate = np.ones(N_Bus)
    non_candidate[ESS_candidate] = 0 # Usefull to constraint the location of ESS storage system to only candidate nodes

    # Constraints: 
    constraints = []
    # Physical limits on power and capacity
    constraints += [Rating_U >= cp.multiply(R_min, U), Rating_U <= cp.multiply(R_max, U)] # Constraint rating to max and min values
    constraints += [Cap_U >= cp.multiply(C_min, U), Cap_U <= cp.multiply(C_max, U)] # Constraint capacity to max and min values
    constraints.append(cp.multiply(non_candidate,U) == np.zeros(N_Bus))
    constraints.append(C_rate * Cap_U >= Rating_U)

    # Benders decompositions 
    bdcut = []  # Multi-cut
    bdcut2 = []  # Single-cut

    for k in range(iter):
        for sc in range(NP):
            bdcut.append(alpha >= obj_2nd[sc, k] + 
                        cp.sum(lambda_2nd[sc, :, :, k], axis=1) @ (Rating_U - previous_rating[:, k]) + 
                        cp.sum(mu_2nd[sc, :, :, k], axis=1) @ (Cap_U - previous_cap[:, k]))

        bdcut2.append(alpha >= cp.sum(obj_2nd[:, k]) + 
                    np.sum(np.sum(lambda_2nd[:, :, :, k], axis=2), axis=0).T @ (Rating_U - previous_rating[:, k]) + 
                    np.sum(np.sum(mu_2nd[:, :, :, k], axis=2), axis=0).T @ (Cap_U - previous_cap[:, k]))

    constraints += bdcut2 + [alpha >= 0] + bdcut

    # Objective function
    objective = (
        cp.multiply(Fixed_cost, cp.sum(U)) + 
        cp.multiply(Power_rating_cost, cp.sum(Rating_U)) + 
        cp.multiply(Energy_capacity_cost, cp.sum(Cap_U)) + 
        alpha
    )

    # Solve
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve(solver=cp.MOSEK)

    Investment = (
        Fixed_cost * np.sum(U.value) + 
        Power_rating_cost * np.sum(Rating_U.value) + 
        Energy_capacity_cost * np.sum(Cap_U.value)
    )
    
    print("finish solving master problem")

    return problem.value, Investment, U.value, Rating_U.value, Cap_U.value, alpha.value

### Incidence_matrices计算

In [5]:
def Incidence_matrices(num_nodes, num_lines, sending_end, receiving_end):
    """
    Create incidence matrices for network topology
    """
    A_plus = np.zeros((num_lines,num_nodes)) # A+ matrix (num_nodes x num_lines)
    for l in range(num_lines):
        A_plus[l,int(sending_end[l])] = 1
        A_plus[l,int(receiving_end[l])] = -1

    A_minus = np.zeros((num_lines,num_nodes)) # A- matrix (num_nodes x num_lines)
    for l in range(num_lines):
        A_minus[l,int(sending_end[l])] = 0
        A_minus[l,int(receiving_end[l])] = -1

    A_plus = np.transpose(A_plus)
    A_minus = np.transpose(A_minus)

    return A_plus, A_minus

### average_cut函数

In [6]:
def average_cut_full(NP, NB, NT, R_sample, obj_val_tilde, lambda_val_tilde, mu_val_tilde):
    """
    Expand sampled dual values and objective values to full arrays,
    using mean for non-sampled scenarios.
    """
    obj_full = np.zeros(NP)
    lambda_full = np.zeros((NP, NB, NT))
    mu_full = np.zeros((NP, NB, NT))

    # Fill sampled
    obj_full[R_sample] = obj_val_tilde
    lambda_full[R_sample, :, :] = lambda_val_tilde
    mu_full[R_sample, :, :] = mu_val_tilde

    # Compute means
    obj_mean = np.mean(obj_val_tilde)
    lambda_mean = np.mean(lambda_val_tilde, axis=0)  # shape (NB, NT)
    mu_mean = np.mean(mu_val_tilde, axis=0)

    # Fill non-sampled with mean
    not_R_sample = np.setdiff1d(np.arange(NP), R_sample)
    obj_full[not_R_sample] = obj_mean
    lambda_full[not_R_sample, :, :] = lambda_mean
    mu_full[not_R_sample, :, :] = mu_mean

    return obj_full, lambda_full, mu_full

### Newcut Function

In [7]:
def Newcut(cb, cb_where):
    if cb_where == GRB.Callback.MIPSOL:
        global cutPoolLazy, cutPoolInfeas
        global cutPoolLazy_added, U_last, Cap_U_last, Rating_U_last, gap_last, objval_last, bnd_last
        global nCuts, countLazy, R_sample, alpha_cur, objVal_cur, objVal_inner
        global U, Cap_U, Rating_U, alpha, U0, Cap_U0, Rating_U0, N_bus, N_line, NP, NT, baseMVA, R_div, Yp
        global R_l, X_l, B_l, K_l, Pd, Qd, pn_bound, qn_bound, v_bound, Pn_solar_bound, G_n, B_n, sending_node, receiving_node
        global a, b, c, freq_scenario
        global m1

        Fixed_cost = 100e3                  # CHF/unit suppose to be e3
        Power_rating_cost = 20000e3         # CHF/p.u. 
        Energy_capacity_cost = 30000e3      # CHF/p.u. 

        if not cutPoolLazy_added:
            if len(cutPoolLazy) > 0:
                print(f"[Newcut] Adding {len(cutPoolLazy)} optimality lazy cuts from previous outer iterations")
                for lazyCut in cutPoolLazy:
                    slope1_sum_over_time = np.sum(lazyCut.slope_1, axis=2)  # shape (NP, N_bus)
                    slope1_final = np.sum(slope1_sum_over_time, axis=0)
                    slope2_sum_over_time = np.sum(lazyCut.slope_2, axis=2)
                    slope2_final = np.sum(slope2_sum_over_time, axis=0)

                    expr = lazyCut.offset[0] + gp.quicksum(slope1_final[e] * Rating_U[e] for e in range(N_bus)) + gp.quicksum(slope2_final[e] * Cap_U[e] for e in range(N_bus))
                    m1.addConstr(alpha >= expr)
            
            cutPoolLazy_added = True

        U_cur = np.zeros(len(U0))
        Cap_U_cur = np.zeros(len(Cap_U0))
        Rating_U_cur = np.zeros(len(Rating_U0))
        for i in range(N_bus):
            U_cur[i] = cb.cbGetSolution(U[i])
            Cap_U_cur[i] = cb.cbGetSolution(Cap_U[i])
            Rating_U_cur[i] = cb.cbGetSolution(Rating_U[i])

        primal_bound = cb.cbGet(GRB.Callback.MIPSOL_OBJBST)
        dual_bound = cb.cbGet(GRB.Callback.MIPSOL_OBJBND)
        objbst = primal_bound
        objbnd = dual_bound
        mip_gap = min(gapcalculate(objbst, objbnd), 1)

        node_count = cb.cbGet(GRB.Callback.MIPSOL_NODCNT)

        if node_count >= 0:
            print(f" [Newcut] Status : {node_count} --- ({np.sum(U_cur)}/{len(U_cur)})")
            U_last = U_cur.copy()
            Cap_U_last = Cap_U_cur.copy()
            Rating_U_last = Rating_U_cur.copy()
            gap_last = mip_gap
            objval_last = objbst
            bnd_last = max(objbnd, bnd_last)

        R_sample = np.sort(np.random.choice(range(NP), size=int(np.ceil(NP/R_div)), replace=False))
        print(f"\t\t\tR_sample : {R_sample}")

        add_infeasibility_cut = False
        nCuts += 1
            
        print(" [Newcut] Calculating Cut")

        IndESS = np.where(U_cur==1)[0]

        ESS_cha_l = np.zeros((N_bus, NT))  # Lower charging limits
        ESS_cha_u = np.zeros((N_bus, NT))  # Upper charging limits
        ESS_cha_u[IndESS, :] = Rating_U_cur[IndESS].reshape(-1, 1)  # Max charging limits

        ESS_dis_l = np.zeros((N_bus, NT))  # Lower discharging limits
        ESS_dis_u = np.zeros((N_bus, NT))  # Upper discharging limits
        ESS_dis_u[IndESS, :] = Rating_U_cur[IndESS].reshape(-1, 1)  # Max discharging limits

        ESS_soc0 = np.zeros((N_bus))
        ESS_soc_l = np.zeros((N_bus, NT))  # Lower SoC limits
        ESS_soc_u = np.zeros((N_bus, NT))  # Upper SoC limits
        ESS_soc_u[IndESS, :] = Cap_U_cur[IndESS].reshape(-1, 1)   # Max SoC limits

        ESS_cha_bound = np.array([ESS_cha_l,ESS_cha_u])
        ESS_dis_bound = np.array([ESS_dis_l,ESS_dis_u])
        ESS_soc_bound = np.array([ESS_soc_l,ESS_soc_u])

        obj, lambda_full, mu_full = run_SOC_ACOPF_for_samples(NP, R_sample, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                              R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                              K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound,
                              Pn_solar_bound, freq_scenario)
        
        add_infeasibility_cut = np.any(np.isnan(obj))
            
        if not add_infeasibility_cut:
            # Objective term
            expr_obj = gp.quicksum(obj[i] for i in range(NP))
            # Lambda term
            expr_lambda = gp.LinExpr()
            for i in range(NP):
                lambda_sum_i = np.sum(lambda_full[i], axis=1)  # (N_bus,)
                expr_lambda += gp.quicksum(lambda_sum_i[e] * (Rating_U[e] - Rating_U_cur[e]) for e in range(N_bus))
            # Mu term
            expr_mu = gp.LinExpr()
            for i in range(NP):
                mu_sum_i = np.sum(mu_full[i], axis=1)  # (N_bus,)
                expr_mu += gp.quicksum(mu_sum_i[e] * (Cap_U[e] - Cap_U_cur[e]) for e in range(N_bus))
            # Final cut
            expr = expr_obj + expr_lambda + expr_mu
            m1.addConstr(alpha >= expr)
            print("\t[Newcut] Adding Slim Cut")

            offset_term = 0.0
            for i in range(NP):
                lambda_sum_i = np.sum(lambda_full[i], axis=1)  # (N_bus,)
                mu_sum_i = np.sum(mu_full[i], axis=1)          # (N_bus,)
                offset_term += np.dot(lambda_sum_i, Rating_U_cur) + np.dot(mu_sum_i, Cap_U_cur)

            offset = [np.sum(obj) - offset_term]

            s0 = PrimalSolution(
                np.where(U_cur > 0)[0].tolist(), 
                U_cur.tolist(),
                np.sum(obj) + Fixed_cost * np.sum(U_cur) + Power_rating_cost * np.sum(Rating_U_cur) + Energy_capacity_cost * np.sum(Cap_U_cur),
                offset,
                lambda_full,
                mu_full, 
                R_sample.tolist()
            )
            cutPoolLazy.append(s0)
            countLazy += 1
            if countLazy % 10 == 0:
                print(f"countlazy = {countLazy}")

        alpha_cur = cb.cbGetSolution(alpha)

        objVal_cur = Fixed_cost * np.sum(U_cur) + Power_rating_cost * np.sum(Rating_U_cur) + Energy_capacity_cost * np.sum(Cap_U_cur) + alpha_cur # lower bound
        objVal_inner = Fixed_cost * np.sum(U_cur) + Power_rating_cost * np.sum(Rating_U_cur) + Energy_capacity_cost * np.sum(Cap_U_cur) + np.sum(obj) # upper bound

        print(f"[Newcut End Logging]\n  --objbst = {objbst}\n  --objbnd = {objbnd}\n  --mip_gap = {mip_gap}\n  --objVal_cur = {objVal_cur}\n  --objVal_inner = {objVal_inner}")

### 子问题求解(grid operation)

#### single scenario

In [8]:
def SOC_ACOPF_2D_alocation(baseMVA, NT, num_nodes, num_lines, Yp, sending_node, receiving_node, IndPV,
               R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n, K_l, quad_cost, lin_cost, const_cost,
               ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound, 
               theta_n_min=-1, theta_n_max=-1, theta_l_min=-1, theta_l_max=-1, eta_dis = 1, eta_cha = 1):

    ######################################################
    """This model is very usefull since it allows to compute the optimal power flow for any grid topology."""
    ######################################################
    """Inputs"""
    # baseMVA : size(1) : Power base in MVA
    # NT : size(1) : Number of time steps
    # num_nodes : size(1) : Number of buses in the grid (NB)
    # num_lines : size(1) : Number of lines in the grid (NL)
    # Yp : size(1) : Planning period
    # sending_node : size(NB) : array with the sending node with number from 0 to N_bus-1 for line l 
    # sending_node : size(NL) : array with the receiving node with number from 0 to N_bus-1 for line l
    # IndPV : Indexes of PV : List where are the PV
    """All values in p.u. except the cost functions [CHF/MW]"""
    # quad_cost : Quadratic cost coefficient
    # lin_cost : Linear cost coefficient
    # const_cost : Constant cost coefficient
    # R_l, X_l, B_l, K_l : size(NL,NT) : r, x, b, ampacity limit for each line at each time step for each line
    # p_d, q_d, G_n, B_n : size(NB,NT) : active power, reactive power, g and b, at each time step for each node
    # pn_bound, qn_bound, v_bound : size(2,NB,NT) : active power, reactive power and voltage magnitude bounds at each time step for each node
    # quad_cost, lin_cost, const_cost : size(NB,NT) : cost of each buses.
    # ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound : size(2,NB,NT) : ESS initial state and bounds for rating and capacity
    ######################################################
    """Output"""
    # problem : size(1) : Total cost of operation
    # p_n, q_n, v_n : size(NB,NT) : active power injection, reactive power injection and voltage magnitude at each time step for each node
    # p_sl, q_sl, p_ol, q_ol, K_ol : size(NL,NT) : active and reactive power flow on lines, active and reactive power flow losses on lines, ampacity losses
    # theta_n, theta_l : size(NB or NL,NT) : angles on buses and lines
    # ESS_soc, ESS_cha, ESS_dis, q_ESS : size(NB,NT) : ESS operation variables 
    # lambda_, mu_ : size(NB,NT) : Dual values of ESS constraints on rating and capacity
    ######################################################
    
    """Initialisation"""
    A_plus, A_minus = Incidence_matrices(num_nodes, num_lines, sending_node, receiving_node)

    # Unfolding nodes data
    p_n_min = pn_bound[0]  # Minimum active power generation at each node
    p_n_max = pn_bound[1]  # Maximum active power generation at each node
    q_n_min = qn_bound[0]  # Minimum reactive power generation at each node
    q_n_max = qn_bound[1]  # Maximum reactive power generation at each node
    V_min = v_bound[0]**2  # Minimum voltage squared at each node
    V_max = v_bound[1]**2  # Maximum voltage squared at each node
    ESS_cha_min = ESS_cha_bound[0] # Minimum charging rate at each node 
    ESS_cha_max = ESS_cha_bound[1] # Maximum charging rate at each node
    ESS_dis_min = ESS_dis_bound[0] # Minimum discharging rate at each node 
    ESS_dis_max = ESS_dis_bound[1] # Maximum discharging rate at each node
    ESS_soc_min = ESS_soc_bound[0] # Minimum state of charge at each node 
    ESS_soc_max = ESS_soc_bound[1] # Maximum state of charge at each node

    # theta_n min and max
    if theta_n_min == -1 : theta_n_min = - np.pi / 2 * np.ones((num_nodes,NT))  # Minimum bus angle 
    if theta_n_max == -1 : theta_n_max = np.pi / 2 * np.ones((num_nodes,NT))  # Maximum bus angle

    # theta_l min and max
    if theta_l_min == -1 : theta_l_min = - np.pi / 2 * np.ones((num_lines,NT))  # Minimum line angle (from relaxation assumption)
    if theta_l_max == -1 : theta_l_max = np.pi / 2 * np.ones((num_lines,NT))  # Maximum line angle (from relaxation assumption)

    # for q_ESS
    lin = 2 # from data
    xx = np.linspace(0, 1/2 * np.pi, lin + 1)
    slope = np.zeros(lin)
    offset = np.zeros(lin)
    for i in range(lin):
        slope[i]=(np.sin(xx[i+1])-np.sin(xx[i]))/(np.cos(xx[i+1])-np.cos(xx[i]))
        offset[i]=(np.sin(xx[i])*np.cos(xx[i+1])-np.sin(xx[i+1])*np.cos(xx[i]))/(np.cos(xx[i+1])-np.cos(xx[i]))

    ######################################################
    """Variables"""
    p_n = cp.Variable((num_nodes,NT))  # Active power at node n
    p_curtailment = cp.Variable((num_nodes, NT), nonneg=True)
    p_slack = p_n[0, :]
    p_imp = cp.pos(p_slack)
    p_exp = cp.pos(-p_slack)
    q_n = cp.Variable((num_nodes,NT))  # Reactive power at node n
    V_n = cp.Variable((num_nodes, NT))  # Voltage magnitude squared at node n
    theta_n = cp.Variable((num_nodes,NT))  # Voltage angles at node n
    p_sl = cp.Variable((num_lines,NT))  # Active power at sending end of line l
    q_sl = cp.Variable((num_lines,NT))  # Reactive power at sending end of line l
    p_ol = cp.Variable((num_lines,NT))  # Active power losses on line l
    q_ol = cp.Variable((num_lines,NT))  # Reactive power losses on line l
    K_ol = cp.Variable((num_lines,NT))  # Branch Equivalent ampacity constraint on line l
    theta_l = cp.Variable((num_lines,NT))  # Voltage angles at line l

    ESS_cha = cp.Variable((num_nodes,NT)) 
    ESS_dis = cp.Variable((num_nodes,NT))
    ESS_soc = cp.Variable((num_nodes,NT))
    q_ESS = cp.Variable((num_nodes,NT))
    
    # Variables for allocation
    Cmax = cp.Variable((num_nodes,NT))
    Rmax = cp.Variable((num_nodes,NT))

    # Create the Incidence Matrices used in the sending end and receiving end voltage
    Inc_sending_cvx = np.zeros((num_lines, num_nodes))
    Inc_receiving_cvx = np.zeros((num_lines, num_nodes))
    for l in range(num_lines):
        Inc_sending_cvx[l, sending_node[l]] = 1
        Inc_receiving_cvx[l, receiving_node[l]] = 1

    ######################################################
    """ Constraints"""
    constraints = []
    objective = 0
    
    ### Bus constraints ###

    # Voltage Magnitude bounds (1k)
    constraints.append(V_n >= V_min)
    constraints.append(V_n <= V_max)

    # Node angle bounds (1m)
    constraints.append(theta_n >= theta_n_min)
    constraints.append(theta_n <= theta_n_max)

    # Active power bounds (1n)
    constraints.append(p_n >= p_n_min)
    constraints.append(p_n <= p_n_max)

    # for time in range(NT):
    #     if len(IndPV[time])>0:
    #         constraints.append(p_n[IndPV[time],:] == p_n_max[IndPV[time],:]) # enforces the power injection to be equal to PV production

    for time in range(NT):
        if len(IndPV[time]) > 0:
            # p_n + p_curtailment = p_n_max
            constraints.append(p_n[IndPV[time], :] + p_curtailment[IndPV[time], :] == p_n_max[IndPV[time], :])

    # Reactive power bounds (1o)
    constraints.append(q_n >= q_n_min)
    constraints.append(q_n <= q_n_max)

    # Alocation related constraints to get the dual variables
    constraint_capacity = Cmax == ESS_soc_max
    constraint_rating = Rmax == ESS_cha_max
    constraints += [constraint_capacity, constraint_rating]

    # ESS charging and discharging rate bounds
    constraints.append(ESS_cha >= ESS_cha_min)
    constraints.append(ESS_cha <= Rmax)
    constraints.append(ESS_dis >= ESS_dis_min)
    constraints.append(ESS_dis <= Rmax)

    # ESS SOC bounds
    constraints.append(ESS_soc >= ESS_soc_min)
    constraints.append(ESS_soc <= Cmax)

    # Linking time steps for ESS (excluding the last time step)
    constraints.append(ESS_soc[:, 1:] == ESS_soc[:, :-1] + ESS_cha[:, :-1] - ESS_dis[:, :-1])
    constraints.append(ESS_soc0 == ESS_soc[:,-1] + ESS_cha[:,-1] - ESS_dis[:,-1]) #last timestep to reset battery charge for next day

    # Initializing ESS SOC for the first time step
    constraints.append(ESS_soc[:, 0] == ESS_soc0)
    constraints.append(ESS_soc[:,-1] == ESS_soc0)
    constraints.append(cp.sum(ESS_cha,axis=1) == cp.sum(ESS_dis,axis=1))

    # Battery aging constraints --> battery has to do at maximum 1.1 cycles per day
    constraints.append(cp.sum(cp.abs(ESS_cha-ESS_dis),axis=1) <= 2*1.1*Cmax[:,0])

    # ESS reactive power computation and bounds
    for i in range(lin):
        constraints.append(q_ESS <= slope[i] * (ESS_cha - ESS_dis) + offset[i] * Rmax)
        constraints.append(q_ESS <= -slope[i] * (ESS_cha - ESS_dis) + offset[i] * Rmax)
        constraints.append(q_ESS >= slope[i] * (ESS_cha - ESS_dis) - offset[i] * Rmax)
        constraints.append(q_ESS >= -slope[i] * (ESS_cha - ESS_dis) - offset[i] * Rmax)

    # Active Power Balance (1b)
    constraints.append(p_n + ESS_dis - Pd - ESS_cha == A_plus @ p_sl - A_minus @ p_ol + cp.multiply(G_n, V_n))

    # Reactive Power Balance (1c)
    constraints.append(q_n - q_ESS - Qd == A_plus @ q_sl - A_minus @ q_ol - cp.multiply(B_n, V_n))

    ### line constraints ###

    # Line angle bounds (1l):
    constraints.append(theta_l >= theta_l_min)
    constraints.append(theta_l <= theta_l_max)

    # Voltage drop constraint (1d):
    constraints.append(Inc_sending_cvx @ V_n - Inc_receiving_cvx @ V_n== 2 * cp.multiply(R_l, p_sl) + 2 * cp.multiply(X_l, q_sl) - cp.multiply(R_l, p_ol) - cp.multiply(X_l, q_ol))
    
    # Conic active and reactive power losses constraint (2b):
    constraints.append(K_ol == cp.multiply((K_l - cp.multiply(Inc_sending_cvx @ V_n, B_l**2) + 2 * cp.multiply(q_sl, B_l)), X_l))
    constraints.append(K_ol >= q_ol)

    # Power loss constraint (2c):
    constraints.append(cp.multiply(p_ol,X_l) == cp.multiply(q_ol,R_l))

    # Line angle constraint (1h):
    constraints.append(theta_l == Inc_sending_cvx @ theta_n - Inc_receiving_cvx @ theta_n)

    # Linearized angle constraint (2d):
    constraints.append(theta_l == cp.multiply(X_l,p_sl) - cp.multiply(R_l,q_sl))

    # Constraints that requiere a loop because of dimensionality limit of cp.norm().
    for time in range(NT):
        for l in range(num_lines):
            # Conic active and reactive power losses constraint (2b): (rest of ineq)
            constraints.append(
                cp.norm(
                    cp.vstack([
                        2 *np.sqrt(X_l[l,time])* cp.vstack([p_sl[l,time], q_sl[l,time]]),
                        cp.reshape(q_ol[l,time] - V_n[sending_node[l],time], (1, 1))
                    ]),2
                ) <= q_ol[l,time] + V_n[sending_node[l],time]
            )

            # Feasibility solution recovery equation (4g):
            constraints.append(
                V_n[sending_node[l],time] + V_n[receiving_node[l],time] >= cp.norm(
                    cp.vstack([
                        2*theta_l[l,time]/np.sin(theta_l_max[l,time]), 
                        V_n[sending_node[l],time] - V_n[receiving_node[l],time]
                    ]), 2)
            )
    

    #####################################################################
    """Objective Function""" 
    curtailment_cost = 26
    price_imp = [620, 620, 620, 620, 620, 620, 620, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 620, 620, 620]
    objective = cp.Minimize(
        Yp * 365 * (cp.sum(cp.multiply(quad_cost, cp.square(p_n * baseMVA)) 
                        + cp.multiply(lin_cost, p_n * baseMVA) 
                       + const_cost)) 
        + Yp * 2910 * cp.sum(ESS_cha + ESS_dis) * baseMVA
        + cp.sum(p_ol) * 100 * 365 * baseMVA * Yp
        + cp.sum(cp.multiply(p_imp, price_imp)) * 365 * baseMVA * Yp
        + cp.sum(p_exp) * 100 *365 *baseMVA * Yp
        + cp.sum(p_curtailment * baseMVA) * curtailment_cost * Yp * 365
    )

    # Defining the optimization problem
    problem = cp.Problem(objective, constraints)
    #####################################################################

    #####################################################################
    # Solve the problem
    problem.solve(solver=cp.MOSEK)

    # Dual values :
    lambda_ = constraint_rating.dual_value # Dual value for the constraint related to ESS maximal rating
    mu_ = constraint_capacity.dual_value # Dual value for the constraint related to ESS maximal capacity

    return problem.value, p_n.value, q_n.value, np.sqrt(V_n.value), p_sl.value, q_sl.value, p_ol.value, q_ol.value, K_ol.value, theta_n.value, theta_l.value, ESS_soc.value, ESS_cha.value, ESS_dis.value, q_ESS.value, lambda_, mu_

In [9]:
def compute_SOC_ACOPF(sc, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                      R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                      K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound, Pn_solar_bound, freq_scenario):

    # Extract time-dependent indices
    idx_PV_sc = [np.where(Pn_solar_bound[sc, 1, :, time] > 0)[0].astype(int) for time in range(NT)]
    Yp_scenario = Yp * freq_scenario

    try:
        print(f"Starting scenario {sc}", flush=True)
        # Call the SOC_ACOPF_2D_alocation function
        # solving sub-problem to get ub and dual solutions
        cost, _, _, _, _, _, _, _, _, _, _, _, _, _, _, lambda_aloc, mu_aloc = SOC_ACOPF_2D_alocation(
            baseMVA, NT, N_bus, N_line, Yp_scenario[sc], sending_node, receiving_node, idx_PV_sc,
            R_l, X_l, B_l, Pd[sc], Qd[sc],
            pn_bound[sc], qn_bound, v_bound,
            G_n, B_n, K_l,
            a, b, c,
            ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound)

        print(f"Finished scenario {sc}: Cost = {cost}", flush=True)
        # Return the results for storage
        return cost, lambda_aloc, mu_aloc

    except Exception as e:
        print(f"Error in compute_SOC_ACOPF for scenario {sc}: {e}", flush=True)
        raise

#### all scenario

In [10]:
def run_SOC_ACOPF_for_samples(NP, R_sample, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                              R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                              K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound,
                              Pn_solar_bound, freq_scenario):
    lambda_list = []
    mu_list = []
    obj_list = []

    for sc in R_sample:
        obj_val, lambda_val, mu_val = compute_SOC_ACOPF(
            sc, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
            R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
            K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound,
            ESS_soc_bound, Pn_solar_bound, freq_scenario
        )
        obj_list.append(obj_val)
        lambda_list.append(lambda_val)
        mu_list.append(mu_val)

    lambda_array = np.stack(lambda_list, axis=0)
    mu_array = np.stack(mu_list, axis=0)
    obj_array = np.array(obj_list)

    obj_full, lambda_full, mu_full = average_cut_full(NP, N_bus, NT, R_sample, obj_array, lambda_array, mu_array)

    return obj_full, lambda_full, mu_full

### 主函数

In [11]:
def main_script():

    ################################################################ Parameters
    ESS_candidate = np.array([8, 18, 25, 33]) #Candidates nodes
    datapath = "Data/" # Change as a function of the loaded folder
    # It is possible to change other vales such as prices and costs detailed later

    ############################################################### Data import
    # Calculate time
    start_time = dt.datetime.now()

    # data for 33-bus system 
    bus_data = pd.read_csv(datapath+"bus_data.csv")
    # bus_data = bus_data.sort_values(by="BUS_I").reset_index(drop=True) # maybe not usefull, to test later
    # bus_index_to_bus_name = bus_data['BUS_I'].to_dict() # dictionary mapping the index number to bus ID
    # bus_name_to_bus_index = {v: k for k, v in bus_index_to_bus_name.items()}
    # bus_data = bus_data.reset_index().rename(columns = {"index":"grid_node"})

    branch_data = pd.read_csv(datapath+"branch_data.csv")
    generator_data = pd.read_csv(datapath+"generator_data.csv")

    # stat_scenario = pd.read_csv(datapath+"Chaudron_scenarios.csv",delimiter=";")
    # freq_scenario = stat_scenario.dp.to_numpy() / stat_scenario.dp.sum()

    # Processed data to get Pd, Qd and PV production per time step and per Scenario
    scenario_data = pd.read_csv(datapath+"Clean_demand.csv")

    # Used as data cleaning since I had one file with N nodes and the other with N-1 nodes
    # I then assumed that the slack was missing so I ajust values to fit 
    # scenario_data.loc[scenario_data[scenario_data.grid_node>=(bus_data[bus_data.BUS_TYPE==3].grid_node).to_list()[0]].index,"grid_node"] += 1 

    # NP = len(scenario_data.Period.unique()[:-2])
    # N_bus = len(bus_data.grid_node.unique())
    # N_line = len(branch_data)
    # NT = 24
    # baseMVA = generator_data.MBASE.max()
    global N_bus, N_line, NP, NT, baseMVA, freq_scenario
    N_bus = len(bus_data.Grid_node.unique())  # 33-bus
    N_line = len(branch_data)
    NP = len(scenario_data.Scenario.unique())  # 12 scenarios
    NT = 24
    baseMVA = 10  # 1e7VA=10MVA

    freq_scenario = np.ones(NP) / NP

    ############################################################## Setting all costs
    # PV generation cost
    quad_cost_PV = 0
    lin_cost_PV = 26
    const_cost_PV = 0

    # Power cost
    # index_slack = (bus_data[bus_data.BUS_TYPE==3].grid_node).to_list()[0]
    index_slack = [0]
    a_slack = np.zeros(N_bus)
    a_slack[index_slack] = 0
    a_slack = a_slack[:, np.newaxis] * np.ones((1, NT))
    b_slack = np.zeros(N_bus)
    b_slack[index_slack] = 200
    b_slack = b_slack[:, np.newaxis] * np.ones((1, NT))
    c_slack = np.zeros(N_bus)
    c_slack[index_slack] = 0
    c_slack = c_slack[:, np.newaxis] * np.ones((1, NT))

    ############################################################## Data preprocessing
    # creating the dataset for values that depends on time and scenario
    # Pd = np.zeros((NP, N_bus, NT))
    # Pn_solar_bound = np.zeros((NP, 2, N_bus, NT))
    # a_PV = np.zeros((N_bus, NT))
    # b_PV = np.zeros((N_bus, NT))
    # c_PV = np.zeros((N_bus, NT))

    # for sc in scenario_data.Period.unique()[:-2]-1:
    #     mask1 = (scenario_data.Period==sc+1)
    #     for time in scenario_data[mask1].Time.unique()-1:
    #         mask2 = (scenario_data.Time==time)
    #         intermediate_df = pd.merge(scenario_data[mask1 & mask2][["grid_node","Domestic_electricity","PV_production"]],
    #             bus_data[["grid_node"]],
    #             on="grid_node",
    #             how="right").fillna(0)
    #         Pd[sc,:,time] = intermediate_df.Domestic_electricity.to_numpy() /1000 /baseMVA # kWh to MW to p.u. 
    #         Pn_solar_bound[sc,1,:,time] = intermediate_df.PV_production.to_numpy() /1000 /baseMVA # kWh to MW to p.u.
    #         a_PV[:,time] = quad_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant
    #         b_PV[:,time] = lin_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant
    #         c_PV[:,time] = const_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant"""

    global Pd, Qd, Pn_solar_bound
    Pd = np.zeros((NP, N_bus, NT))
    Qd = np.zeros((NP, N_bus, NT))
    Pn_solar_bound = np.zeros((NP, 2, N_bus, NT))
    index_PV = [16, 31]
    a_PV = np.zeros((N_bus, NT))
    b_PV = np.zeros((N_bus, NT))
    c_PV = np.zeros((N_bus, NT))

    for sc in range(1, NP + 1):
        for time in range(1, NT + 1):
            mask = (scenario_data.Scenario == sc) & (scenario_data.Time == time)
            intermediate_df = pd.merge(
                scenario_data[mask][["Grid_node", "Pd", "Qd", "PV"]],
                pd.DataFrame({"Grid_node": np.arange(1, N_bus + 1)}),
                on="Grid_node",
                how="right"
            ).fillna(0)

            Pd[sc-1, :, time-1] = intermediate_df.Pd.to_numpy() / 1000 / baseMVA  # kW to p.u.
            Qd[sc-1, :, time-1] = intermediate_df.Qd.to_numpy() / 1000 / baseMVA  # kW to p.u.
            Pn_solar_bound[sc-1, 0, :, time-1] = 0
            Pn_solar_bound[sc-1, 1, :, time-1] = intermediate_df.PV.to_numpy() / 1000 / baseMVA  # kW to p.u.

    a_PV[index_PV, :] = quad_cost_PV
    b_PV[index_PV, :] = lin_cost_PV
    c_PV[index_PV, :] = const_cost_PV

    # expanding dataset for all other data that is not time or scenario dependant
    V_base = bus_data.baseKV.max()  # 12.66 kV
    Z_base = (V_base ** 2) / baseMVA  # kV^2 / MVA = 16.02756 Ohm

    # related to bus data
    # q_d = bus_data.Qd.to_numpy() /baseMVA /5 #reactive power demand
    # q_d= q_d[:, np.newaxis] * np.ones((1, NT))
    global v_bound, G_n, B_n
    G_n = bus_data.Gs.to_numpy() * Z_base
    G_n = G_n[:, np.newaxis] * np.ones((1, NT))
    B_n = bus_data.Bs.to_numpy() * Z_base
    B_n = B_n[:, np.newaxis] * np.ones((1, NT))
    v_bound = bus_data[["Vmin","Vmax"]].to_numpy()
    v_bound = v_bound.T[:, :, np.newaxis] * np.ones((1, N_bus, NT))

    # related to generator data
    # pn_bound = np.zeros((N_bus,2))
    # qn_bound = np.zeros((N_bus,2))
    # index_gen = generator_data["GEN_BUS"].map(bus_name_to_bus_index).to_numpy()
    # generator_data["bus_index"] = index_gen
    # for _, row in generator_data.iterrows():
    #     index = row["bus_index"]
    #     pn_bound[index,0] = -row["PMAX"] /baseMVA 
    #     pn_bound[index,1] = row["PMAX"] /baseMVA 
    #     qn_bound[index,0] = row["QMIN"] /baseMVA 
    #     qn_bound[index,1] = row["QMAX"] /baseMVA 
    # pn_bound = pn_bound.T[np.newaxis,:, :, np.newaxis] * np.ones((NP, 1, N_bus, NT))
    # pn_bound += Pn_solar_bound #Add the PV generation to other generation
    # qn_bound = qn_bound.T[:, :, np.newaxis] * np.ones((1, N_bus, NT))

    global pn_bound, qn_bound
    # if no generator, only contains PV bound
    pn_bound = np.zeros((N_bus, 2))
    qn_bound = np.zeros((N_bus, 2)) 

    slack_Pmax = 1e4  # choose a large headroom in MW
    slack_Pmin = -1e4
    slack_Qmax = 1e4
    slack_Qmin = -1e4
    
    pn_bound[index_slack, 0] = slack_Pmin / baseMVA
    pn_bound[index_slack, 1] = slack_Pmax / baseMVA
    qn_bound[index_slack, 0] = slack_Qmin / baseMVA
    qn_bound[index_slack, 1] = slack_Qmax / baseMVA

    index_gen = (generator_data["GEN_BUS"].astype(int) - 1).values
    # generator_data["bus_index"] = index_gen
    a_gen = np.zeros(N_bus)
    a_gen[index_gen] = [0.12, 0.09]
    a_gen = a_gen[:, np.newaxis] * np.ones((1, NT))
    b_gen = np.zeros(N_bus)
    b_gen[index_gen] = [20, 15]
    b_gen = b_gen[:, np.newaxis] * np.ones((1, NT))
    c_gen = np.zeros(N_bus)
    c_gen[index_gen] = [0, 0]
    c_gen = c_gen[:, np.newaxis] * np.ones((1, NT))

    pn_bound[index_gen, 0] = generator_data["P_min"].values / baseMVA
    pn_bound[index_gen, 1] = generator_data["P_max"].values / baseMVA
    qn_bound[index_gen, 0] = generator_data["Q_min"].values / baseMVA
    qn_bound[index_gen, 1] = generator_data["Q_max"].values / baseMVA

    pn_static = pn_bound.T[np.newaxis, :, :, np.newaxis]  # (1,2,N_bus,1)
    qn_static = qn_bound.T[:, :, np.newaxis]  # (2,N_bus,1)

    # 2) Broadcast to (NP,2,N_bus,NT)
    pn_bound = pn_static * np.ones((NP, 2, N_bus, NT))

    # 3) Finally add the PV bounds
    pn_bound += Pn_solar_bound  # both are now (NP,2,N_bus,NT)
    qn_bound = qn_static * np.ones((2, N_bus, NT))

    # related to branch data
    # sending_node = branch_data["F_BUS"].map(bus_name_to_bus_index).to_numpy().astype(int)
    global sending_node, receiving_node, R_l, X_l, B_l, K_l
    # receiving_node = branch_data["T_BUS"].map(bus_name_to_bus_index).to_numpy().astype(int)
    sending_node = branch_data["F_BUS"].to_numpy().astype(int) - 1  # converted to 0-based
    receiving_node = branch_data["T_BUS"].to_numpy().astype(int) - 1  # converted to 0-based
    R_l = branch_data["BR_R"].to_numpy() / Z_base
    R_l = R_l[:, np.newaxis] * np.ones((1, NT))
    X_l = branch_data["BR_X"].to_numpy() / Z_base
    X_l = X_l[:, np.newaxis] * np.ones((1, NT))
    B_l = branch_data["BR_B"].to_numpy() * Z_base
    B_l = B_l[:, np.newaxis] * np.ones((1, NT))
    K_l = branch_data["Pup"].to_numpy() / 1000 / baseMVA  # # Ampacity for each line (kW to p.u.)
    K_l = K_l[:, np.newaxis] * np.ones((1, NT))
    # K_l = 1 * np.ones(len(branch_data))  # Ampacity for each line
    # K_l = K_l[:, np.newaxis] * np.ones((1, NT))

    # ESS candidate to index
    # ESS_candidate = np.vectorize(bus_name_to_bus_index.get)(ESS_candidate).astype(int)
    ESS_candidate -= 1
    ESS_soc0 = np.zeros((N_bus))

    global a, b, c
    # summing all cost
    a = a_slack + a_PV + a_gen
    b = b_slack + b_PV + b_gen
    c = c_slack + c_PV + c_gen

    ###################################################################### Allocations constaints
    R_min = 0.05 / baseMVA * np.ones(N_bus)
    R_max = 4 / baseMVA * np.ones(N_bus)
    R_bounds = np.array([R_min,R_max])
    C_min = 0.1 / baseMVA * np.ones(N_bus)
    C_max = 7 /baseMVA * np.ones(N_bus)
    C_bounds = np.array([C_min,C_max])
    Fixed_cost = 100e3                  # CHF/unit suppose to be e3
    Power_rating_cost = 20000e3         # CHF/p.u. 
    Energy_capacity_cost = 30000e3      # CHF/p.u. 

    ################################################################# MILP
    global Yp
    Yp = 10
    C_rate = 1

    lb = np.zeros(N_bus)
    ub = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]
    
    global U, Cap_U, Rating_U, R_div
    R_div = 4

    # SETUP TRACKING VARIABLES
    global bnd_last
    confidence_adjusted_bound_gap = float('inf')
    U_last = np.zeros(N_bus)
    Cap_U_last = np.zeros(N_bus)
    Rating_U_last = np.zeros(N_bus)
    gap_last = 1
    objval_last = 1
    bnd_last = -1
    objtrue_iters = 0 

    global nCuts, U0, Cap_U0, Rating_U0
    nCuts = 0 # for logging purposes only

    print(f"[SCPNetworkOpt] Configuration:\n\t\t\t \
        \t\tSampling: {math.ceil(NP/R_div)}/{NP} Days at every iteration (R_div = {R_div}, NP = {NP}")
    
    objs = np.zeros(NP)

    # INITIALIZE MODEL
    global m1
    m1 = gp.Model("optimization_model")
    
    m1.setParam("OutputFlag", 1)
    m1.setParam("LazyConstraints", 1)
    m1.setParam("MIPGap", 0.01)

    U = {}
    Cap_U = {}
    Rating_U = {}
    for e in range(N_bus):
        U[e] = m1.addVar(vtype=GRB.BINARY, name=f"U_{e}")
        Cap_U[e] = m1.addVar(vtype=GRB.CONTINUOUS, name=f"Cap_U{e}")
        Rating_U[e] = m1.addVar(vtype=GRB.CONTINUOUS, name=f"Rating_U{e}")

    #Values that need to be calculated
    non_candidate = np.ones(N_bus)
    non_candidate[ESS_candidate] = 0 # Usefull to constraint the location of ESS storage system to only candidate nodes

    # Add constraints
    U0 = lb.copy()
    Cap_U0 = lb.copy()
    Rating_U0 = lb.copy()

    for e in range(N_bus):
        m1.addConstr(U[e] >= lb[e], name=f"lb_constraint_{e}")
        m1.addConstr(U[e] <= ub[e], name=f"ub_constraint_{e}")
        U[e].start = ub[e]

    for e in range(N_bus):
        m1.addConstr(Rating_U[e] >= float(R_min[e]) * U[e], name=f"rating_min_{e}")
        m1.addConstr(Rating_U[e] <= float(R_max[e]) * U[e], name=f"rating_max_{e}")
        # Rating_U[e].start = lb[e]

        m1.addConstr(Cap_U[e] >= float(C_min[e]) * U[e], name=f"cap_min_{e}")
        m1.addConstr(Cap_U[e] <= float(C_max[e]) * U[e], name=f"cap_max_{e}")
        # Cap_U[e].start = lb[e]

        m1.addConstr(C_rate * Cap_U[e] >= Rating_U[e], name=f"c_rate_{e}")

        if e not in ESS_candidate:
            m1.addConstr(U[e] == 0, name=f"not_candidate_{e}")

    # PERFORM 1 DETERMINISTIC CUT IN THE BEGINNING
    obj0 = np.nan
    add_infeasibility_cut_det = False
    global alpha

    alpha = m1.addVar(lb=0, name="alpha")

    # LB
    m1.setObjective(
        quicksum(Fixed_cost * U[e] for e in range(N_bus)) +
        quicksum(Power_rating_cost * Rating_U[e] for e in range(N_bus)) +
        quicksum(Energy_capacity_cost * Cap_U[e] for e in range(N_bus)) +
        alpha,
        GRB.MINIMIZE
    )

    IndESS0 = np.where(U0 == 1)[0]

    # Charging limits
    ESS_cha_l0 = np.zeros((N_bus, NT))  # Lower charging limits
    ESS_cha_u0 = np.zeros((N_bus, NT))  # Upper charging limits
    ESS_cha_u0[IndESS0, :] = Rating_U0[IndESS0].reshape(-1, 1)  # Max charging limits
    # Discharging limits
    ESS_dis_l0 = np.zeros((N_bus, NT))  # Lower discharging limits
    ESS_dis_u0 = np.zeros((N_bus, NT))  # Upper discharging limits
    ESS_dis_u0[IndESS0, :] = Rating_U0[IndESS0].reshape(-1, 1)  # Max discharging limits
    # State of Charge (SoC)
    ESS_soc00 = np.zeros((N_bus))
    ESS_soc_l0 = np.zeros((N_bus, NT))  # Lower SoC limits
    ESS_soc_u0 = np.zeros((N_bus, NT))  # Upper SoC limits
    ESS_soc_u0[IndESS0, :] = Cap_U0[IndESS0].reshape(-1, 1)   # Max SoC limits

    ESS_cha_bound0 = np.array([ESS_cha_l0,ESS_cha_u0])
    ESS_dis_bound0 = np.array([ESS_dis_l0,ESS_dis_u0])
    ESS_soc_bound0 = np.array([ESS_soc_l0,ESS_soc_u0])

    obj0, lambda0, mu0 = run_SOC_ACOPF_for_samples(NP, range(NP), NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                              R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                              K_l, a, b, c, ESS_soc00, ESS_cha_bound0, ESS_dis_bound0, ESS_soc_bound0,
                              Pn_solar_bound, freq_scenario)
    add_infeasibility_cut_det = np.isnan(obj0).any()

    if not add_infeasibility_cut_det:  # optimality cut
        expr_lambda = gp.LinExpr()
        for i in range(NP):
            lambda_sum_i = np.sum(lambda0[i], axis=1)  # sum over NT, get (N_bus,)
            expr_lambda += gp.quicksum(lambda_sum_i[e] * (Rating_U[e] - Rating_U0[e]) for e in range(N_bus))
        expr_mu = gp.LinExpr()
        for i in range(NP):
            mu_sum_i = np.sum(mu0[i], axis=1)  # sum over NT
            expr_mu += gp.quicksum(mu_sum_i[e] * (Cap_U[e] - Cap_U0[e]) for e in range(N_bus))
        m1.addConstr(alpha >= np.sum(obj0) + expr_lambda + expr_mu, name="optimality_cut_det")

    if add_infeasibility_cut_det:
        print("\t\t[Deterministic Cut @ Beginning] Adding infeasibility cut")
        m1.addConstr(
            quicksum((1 - U0[e]) * U[e] for e in range(N_bus)) >= 1,
            name="infeas_cut_1"
        )
        m1.addConstr(
            quicksum(U0[e] * (1 - U[e]) for e in range(N_bus)) + 
            quicksum((1 - U0[e]) * U[e] for e in range(N_bus)) >= 1,
            name="infeas_cut_2"
        )

    ##################################################################################################################
    # Outer approximation method for Convex Integer Optimization (CIO)
    global cutPoolLazy_added, cutPoolLazy, cutPoolInfeas, countLazy
    cutPoolLazy_added = False
    cutPoolLazy = []
    cutPoolInfeas = []
    countLazy = 0

    time_setup = dt.datetime.now() - start_time

    # Solve the model and get the optimal solutions
    objtrue_iters = 0
    objtrue_maxiters = 20

    objtrue = -1
    objgaptrue = 1
    objgap = 1
    objval = float('-inf')
    Uopt = np.zeros(N_bus)
    objbnd = -1
    confidence_adjusted_bound_gap = 1

    for iter in range(objtrue_maxiters):
        cutPoolLazy_added = False
        m1.optimize(callback=Newcut)

        objtrue = -1
        objgaptrue = 1
        objgap = 1
        objval = float("-inf")

        termination_code = m1.Status
        terminated_ok = (termination_code == GRB.OPTIMAL)

        print(f"terminated with values: {terminated_ok} --- termination_status(m1) = {termination_code}")

        if terminated_ok:
            print("IN TERMINATED_OK PART")

            Uopt = np.array([U[e].X for e in range(N_bus)])
            Rating_Uopt = np.array([Rating_U[e].X for e in range(N_bus)])
            Cap_Uopt = np.array([Cap_U[e].X for e in range(N_bus)])

            objval = m1.ObjVal
            objbnd = m1.ObjBound
            objgap = abs(objval - objbnd) / abs(objval) if abs(objval) > 1e-6 else 1.0

            IndESStrue = np.where(Uopt == 1)[0]
            ESS_cha_ltrue = np.zeros((N_bus, NT))  # Lower charging limits
            ESS_cha_utrue = np.zeros((N_bus, NT))  # Upper charging limits
            ESS_cha_utrue[IndESStrue, :] = Rating_Uopt[IndESStrue].reshape(-1, 1)  # Max charging limits
            ESS_dis_ltrue = np.zeros((N_bus, NT))  # Lower discharging limits
            ESS_dis_utrue = np.zeros((N_bus, NT))  # Upper discharging limits
            ESS_dis_utrue[IndESStrue, :] = Rating_Uopt[IndESStrue].reshape(-1, 1)  # Max discharging limits
            ESS_soc0true = np.zeros((N_bus))
            ESS_soc_ltrue = np.zeros((N_bus, NT))  # Lower SoC limits
            ESS_soc_utrue = np.zeros((N_bus, NT))  # Upper SoC limits
            ESS_soc_utrue[IndESStrue, :] = Cap_Uopt[IndESStrue].reshape(-1, 1)   # Max SoC limits
            ESS_cha_boundtrue = np.array([ESS_cha_ltrue,ESS_cha_utrue])
            ESS_dis_boundtrue = np.array([ESS_dis_ltrue,ESS_dis_utrue])
            ESS_soc_boundtrue = np.array([ESS_soc_ltrue,ESS_soc_utrue])
            objtrue, _, _ = run_SOC_ACOPF_for_samples(
                NP, np.arange(NP), NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                K_l, a, b, c, ESS_soc0true, ESS_cha_boundtrue, ESS_dis_boundtrue, ESS_soc_boundtrue,
                Pn_solar_bound, freq_scenario
                )
            objtrue = np.sum(objtrue) + Fixed_cost * np.sum(Uopt) + Power_rating_cost * np.sum(Rating_Uopt) + Energy_capacity_cost * np.sum(Cap_Uopt)
            objgaptrue = gapcalculate(objtrue, objbnd)

            print(f"[Outer Loop] Finished Iteration {iter}:")
            print(f"\t\tObjval = {objval} - Objbnd = {objbnd} - Objgap = {objgap} - Objtrue = {objtrue} - Objgaptrue = {objgaptrue}")
        else:
            print("IN THE OTHER SIDE")

            print(f"U_Last has dim: {U_last.shape if hasattr(U_last, 'shape') else len(U_last)}")

            Uopt = U_last.copy()
            Cap_Uopt = Cap_U_last.copy()
            Rating_Uopt = Rating_U_last.copy()
            objval = objval_last
            objbnd = bnd_last
            objgap = gap_last

            IndESStrue = np.where(Uopt == 1)[0]
            ESS_cha_ltrue = np.zeros((N_bus, NT))  # Lower charging limits
            ESS_cha_utrue = np.zeros((N_bus, NT))  # Upper charging limits
            ESS_cha_utrue[IndESStrue, :] = Rating_Uopt[IndESStrue].reshape(-1, 1)  # Max charging limits
            ESS_dis_ltrue = np.zeros((N_bus, NT))  # Lower discharging limits
            ESS_dis_utrue = np.zeros((N_bus, NT))  # Upper discharging limits
            ESS_dis_utrue[IndESStrue, :] = Rating_Uopt[IndESStrue].reshape(-1, 1)  # Max discharging limits
            ESS_soc0true = np.zeros((N_bus))
            ESS_soc_ltrue = np.zeros((N_bus, NT))  # Lower SoC limits
            ESS_soc_utrue = np.zeros((N_bus, NT))  # Upper SoC limits
            ESS_soc_utrue[IndESStrue, :] = Cap_Uopt[IndESStrue].reshape(-1, 1)   # Max SoC limits
            ESS_cha_boundtrue = np.array([ESS_cha_ltrue,ESS_cha_utrue])
            ESS_dis_boundtrue = np.array([ESS_dis_ltrue,ESS_dis_utrue])
            ESS_soc_boundtrue = np.array([ESS_soc_ltrue,ESS_soc_utrue])
            objtrue, _, _ = run_SOC_ACOPF_for_samples(
                NP, np.arange(NP), NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                K_l, a, b, c, ESS_soc0true, ESS_cha_boundtrue, ESS_dis_boundtrue, ESS_soc_boundtrue,
                Pn_solar_bound, freq_scenario
                )
            objtrue = np.sum(objtrue) + Fixed_cost * np.sum(Uopt) + Power_rating_cost * np.sum(Rating_Uopt) + Energy_capacity_cost * np.sum(Cap_Uopt)
            objgaptrue = gapcalculate(objtrue, objbnd)

            print("[Outer Loop] Model timed out, but last U is feasible")
            print(f" --- objgap = {objgap}, objbnd = {objbnd}, objtrue = {objtrue}, objgaptrue = {objgaptrue}")
        print("Keeping on")

        # ################# 
        # CONFIDENCE EXIT
        # ################# 
    
        confidence_level = 0.90
        Z = norm.ppf((1 + confidence_level) / 2)  # ~1.64

        global R_sample
        W = len(R_sample)
        print(f"W length is {W}")

        IndESSs = np.where(Uopt == 1)[0]
        ESS_cha_ls = np.zeros((N_bus, NT))  # Lower charging limits
        ESS_cha_us = np.zeros((N_bus, NT))  # Upper charging limits
        ESS_cha_us[IndESSs, :] = Rating_Uopt[IndESSs].reshape(-1, 1)  # Max charging limits
        ESS_dis_ls = np.zeros((N_bus, NT))  # Lower discharging limits
        ESS_dis_us = np.zeros((N_bus, NT))  # Upper discharging limits
        ESS_dis_us[IndESSs, :] = Rating_Uopt[IndESSs].reshape(-1, 1)  # Max discharging limits
        ESS_soc0s = np.zeros((N_bus))
        ESS_soc_ls = np.zeros((N_bus, NT))  # Lower SoC limits
        ESS_soc_us = np.zeros((N_bus, NT))  # Upper SoC limits
        ESS_soc_us[IndESSs, :] = Cap_Uopt[IndESSs].reshape(-1, 1)   # Max SoC limits
        ESS_cha_bounds = np.array([ESS_cha_ls,ESS_cha_us])
        ESS_dis_bounds = np.array([ESS_dis_ls,ESS_dis_us])
        ESS_soc_bounds = np.array([ESS_soc_ls,ESS_soc_us])
        objs, _, _ = run_SOC_ACOPF_for_samples(
            NP, np.arange(NP), NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
            R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
            K_l, a, b, c, ESS_soc0s, ESS_cha_bounds, ESS_dis_bounds, ESS_soc_bounds,
            Pn_solar_bound, freq_scenario
            )

        U_tilde = Fixed_cost * np.sum(Uopt) + Power_rating_cost * np.sum(Rating_Uopt) + Energy_capacity_cost * np.sum(Cap_Uopt) + (NP / W) * np.sum(objs[R_sample])
        s_U = np.sqrt(1 / (W - 1) * np.sum((objs[R_sample] - np.mean(objs[R_sample])) ** 2))

        upper_confidence_bound = U_tilde + Z * s_U / np.sqrt(W)
        lower_confidence_bound = U_tilde - Z * s_U / np.sqrt(W)

        # Calculate the confidence-adjusted bound gap
        confidence_adjusted_bound_gap = (upper_confidence_bound - objval) / upper_confidence_bound
        e_parameter = -0.0001

        print(f"[Outer Loop] Checking Confidence Bound for Termination... \n\tObjgap: {objgap}, ObjGapTrue: {objgaptrue} ---")
        print(f"\tObj: {objval}, ObjTrue: {objtrue}")

        # Check if termination condition is met
        if (confidence_adjusted_bound_gap < e_parameter) or (R_div == 1):
            print(f"[Outer Loop] Terminate the optimization process after {objtrue_iters+1} outer iterations.")
            print(f"\t Termination Conditions: \n\t\tconfidence_adjusted_bound_gap = {confidence_adjusted_bound_gap}, R_div = {R_div}")
            print(f"\tupper_confidence_bound: {upper_confidence_bound}, objval: {objval}, confidence_adjusted_bound_gap = {confidence_adjusted_bound_gap}")
            print(f"\tU_tilde: {U_tilde}, s_U: {s_U}, mean(objs[R_sample]) = {np.mean(objs[R_sample])}, mean(objs) = {np.mean(objs)}")
            break

        print("---> NOT BREAKING ---")
        print(f"     upper_confidence_bound: {upper_confidence_bound}, objval: {objval}, confidence_adjusted_bound_gap = {confidence_adjusted_bound_gap}")
        print(f"     U_tilde: {U_tilde}, s_U: {s_U}, mean(objs[R_sample]) = {np.mean(objs[R_sample])}, mean(objs) = {np.mean(objs)}")
        print(f"     Objgap: {objgap}, ObjGapTrue: {objgaptrue}")
        print(f"     obj true ITERS: {objtrue_iters} / {objtrue_maxiters}")
        print("<--- ")

        # ADD ONE DETERMINISTIC CUT AT LAST SOLUTION
        IndESS0 = np.where(Uopt == 1)[0]
        ESS_cha_l0 = np.zeros((N_bus, NT))  # Lower charging limits
        ESS_cha_u0 = np.zeros((N_bus, NT))  # Upper charging limits
        ESS_cha_u0[IndESS0, :] = Rating_Uopt[IndESS0].reshape(-1, 1)  # Max charging limits
        ESS_dis_l0 = np.zeros((N_bus, NT))  # Lower discharging limits
        ESS_dis_u0 = np.zeros((N_bus, NT))  # Upper discharging limits
        ESS_dis_u0[IndESS0, :] = Rating_Uopt[IndESS0].reshape(-1, 1)  # Max discharging limits
        ESS_soc00 = np.zeros((N_bus))
        ESS_soc_l0 = np.zeros((N_bus, NT))  # Lower SoC limits
        ESS_soc_u0 = np.zeros((N_bus, NT))  # Upper SoC limits
        ESS_soc_u0[IndESS0, :] = Cap_Uopt[IndESS0].reshape(-1, 1)   # Max SoC limits
        ESS_cha_bound0 = np.array([ESS_cha_l0,ESS_cha_u0])
        ESS_dis_bound0 = np.array([ESS_dis_l0,ESS_dis_u0])
        ESS_soc_bound0 = np.array([ESS_soc_l0,ESS_soc_u0])
        obj0, lambda0, mu0 = run_SOC_ACOPF_for_samples(
            NP, np.arange(NP), NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
            R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
            K_l, a, b, c, ESS_soc00, ESS_cha_bound0, ESS_dis_bound0, ESS_soc_bound0,
            Pn_solar_bound, freq_scenario
            )
        expr_lambda = gp.LinExpr()
        for i in range(NP):
            lambda_sum_i = np.sum(lambda0[i], axis=1)  # (N_bus,)
            expr_lambda += gp.quicksum(lambda_sum_i[e] * (Rating_U[e] - Rating_Uopt[e]) for e in range(N_bus))
        expr_mu = gp.LinExpr()
        for i in range(NP):
            mu_sum_i = np.sum(mu0[i], axis=1)  # (N_bus,)
            expr_mu += gp.quicksum(mu_sum_i[e] * (Cap_U[e] - Cap_Uopt[e]) for e in range(N_bus))
        expr = np.sum(obj0) + expr_lambda + expr_mu
        m1.addConstr(alpha >= expr, name="deterministic_cut")

        # ADD LAZY CUTS AS CONSTRAINTS
        if len(cutPoolLazy) > 0:
            cLazy = cutPoolLazy[-1]  # Last added cut
            slope1_sum_over_time = np.sum(cLazy.slope_1, axis=2)  # shape (NP, N_bus)
            slope1_final = np.sum(slope1_sum_over_time, axis=0)
            slope2_sum_over_time = np.sum(cLazy.slope_2, axis=2)
            slope2_final = np.sum(slope2_sum_over_time, axis=0)
            expr = cLazy.offset[0] + gp.quicksum(slope1_final[e] * Rating_U[e] for e in range(N_bus)) + gp.quicksum(slope2_final[e] * Cap_U[e] for e in range(N_bus))

            m1.addConstr(alpha >= expr, name="last_lazy_cut_as_normal_constraint")

        # UPDATE R_div
        R_div = max(int(np.ceil(R_div / 1.5)), 1)
        print(f"New R_div = {R_div}")

        # UPDATE ITERATION COUNTER AND TIME LIMIT
        objtrue_iters += 1               

    time_scp = dt.datetime.now() - start_time - time_setup
    return Uopt, Cap_Uopt, Rating_Uopt, objtrue, objval, objbnd, objgaptrue, objgap, time_scp, confidence_adjusted_bound_gap, objtrue_iters        

### results

In [12]:
Uopt_scp, Cap_Uopt_scp, Rating_Uopt_scp, objtrue, objval_scp, objbnd, gaptrue_scp, objgap_scp, time_scp, confidence_adjusted_bound_gap, objtrue_iters = main_script()

print("\n------ Final Results ------")
print(f"Total Time (SCP): {time_scp.total_seconds():.2f} s")
print(f"Objval (SCP relaxed): {objval_scp}")
print(f"Objtrue (original):   {objtrue}")
print(f"Gap: {objgap_scp}, True Gap: {gaptrue_scp}")
print(f"Confidence-Adjusted Bound Gap: {confidence_adjusted_bound_gap}")
print(f"True Outer Loop Iters: {objtrue_iters+1}")

print("\nSelected U (ESS location):")
print(np.round(Uopt_scp, 3))
print("\nCap_U (ESS capacity):")
print(Cap_Uopt_scp)
print("\nRating_U (ESS Rating):")
print(Rating_Uopt_scp)

[SCPNetworkOpt] Configuration:
			         		Sampling: 3/12 Days at every iteration (R_div = 4, NP = 12
Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-25
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0.01
Starting scenario 0
Finished scenario 0: Cost = 2575721.925365602
Starting scenario 1
Finished scenario 1: Cost = 1243746.8751153331
Starting scenario 2
Finished scenario 2: Cost = 2796376.289168246
Starting scenario 3
Finished scenario 3: Cost = 1485128.6216989225
Starting scenario 4
Finished scenario 4: Cost = 2471632.7655365933
Starting scenario 5
Finished scenario 5: Cost = 916296.7952973881
Starting scenario 6
Finished scenario 6: Cost = 2754801.4415139295
Starting scenario 7
Finished scenario 7: Cost = 990084.2331819732
Starting scenario 8
Finished scenario 8: Cost = 3195570.5295187794
Starting scenario 9
Finished scenario 9: Cost = 1612071.6273115391
Starting scenario 10
Finished scenario 10: Cost = 3313958.56964

GurobiError: Problem adding constraints

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\uji_crush\AppData\Local\Temp\ipykernel_14828\3786200852.py", line 103, in Newcut
  File "src\\gurobipy\\model.pxi", line 3794, in gurobipy.Model.addConstr
  File "src\\gurobipy\\model.pxi", line 3478, in gurobipy.Model.__addConstr
gurobipy.GurobiError: Problem adding constraints


User MIP start produced solution with objective 3.48621e+07 (20.49s)
 [Newcut] Status : 0.0 --- (3.0/33)
			R_sample : [4 6 7]
 [Newcut] Calculating Cut
Starting scenario 4
Finished scenario 4: Cost = 2352924.4583369796
Starting scenario 6
Finished scenario 6: Cost = 2633376.3308466645
Starting scenario 7
Finished scenario 7: Cost = 880239.5183660743


GurobiError: Problem adding constraints

...

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\uji_crush\AppData\Local\Temp\ipykernel_14828\3786200852.py", line 81, in Newcut
  File "C:\Users\uji_crush\AppData\Local\Temp\ipykernel_14828\314278818.py", line 10, in run_SOC_ACOPF_for_samples
  File "C:\Users\uji_crush\AppData\Local\Temp\ipykernel_14828\1122046912.py", line 13, in compute_SOC_ACOPF
  File "C:\Users\uji_crush\AppData\Local\Temp\ipykernel_14828\1013406347.py", line 242, in SOC_ACOPF_2D_alocation
  File "d:\Users\uji_crush\anaconda3\envs\cvxpy_env\Lib\site-packages\cvxpy\problems\problem.py", line 503, in solve
    return solve_func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\uji_crush\anaconda3\envs\cvxpy_env\Lib\site-packages\cvxpy\problems\problem.py", line 1073, in _solve
    data, solving_chain, inverse_data = self.get_problem_data(
             

Found heuristic solution: objective 3.336211e+07

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5210663e+07   0.000000e+00   0.000000e+00     53s

Root relaxation: objective 2.521066e+07, 0 iterations, 0.00 seconds (0.00 work units)
 [Newcut] Status : 0.0 --- (0.0/33)
			R_sample : [1 7 8]
 [Newcut] Calculating Cut
Starting scenario 1
Finished scenario 1: Cost = 1243746.8751153331
Starting scenario 7


: 

: 